In [ ]:
!pip install opendatasets

In [ ]:
dataset_link = 'https://www.kaggle.com/datasets/enesyurt/tuav-fire-detection'

In [ ]:
import opendatasets as od

In [ ]:
od.download(dataset_link)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jahnvisahni
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/enesyurt/tuav-fire-detection


100%|██████████| 1.15G/1.15G [00:11<00:00, 104MB/s]


## IMPORTS

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import cv2
import shutil
from shutil import copyfile
import random

import keras
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator # from keras.preprocessing.image import ImageDataGenerator is deprecated
from sklearn.model_selection import train_test_split
from keras.utils import load_img, img_to_array
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from keras.applications import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, MaxPooling2D, BatchNormalization, Activation, Dropout, Flatten, Conv2D
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### PLOT HISTORY

### Setting up the data:

In [ ]:
data_dir = './tuav-fire-detection'
os.listdir(data_dir)

['Member_7',
 'Member_3',
 'Member_1',
 'Member_9',
 'Member_2',
 'Member_6',
 'Member_4',
 'Member_8',
 'Member_5']

In [30]:
train_dir = "new_dataset1/train"
test_dir = "new_dataset1/test"

# Create train and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define the split ratio (e.g., 80/20)
split_ratio = 0.8

# Loop through each member folder
for member_folder in os.listdir(data_dir):
    member_path = os.path.join(data_dir, member_folder)
    if not os.path.isdir(member_path):
        continue  # Skip if not a directory

    # Create member directories in train and test directories
    train_member_dir = os.path.join(train_dir, member_folder)
    test_member_dir = os.path.join(test_dir, member_folder)
    os.makedirs(train_member_dir, exist_ok=True)
    os.makedirs(test_member_dir, exist_ok=True)

    # Get the list of images
    images = os.listdir(member_path)

    # Shuffle the images randomly
    random.shuffle(images)

    # Calculate the split index based on the split ratio
    split_index = int(len(images) * split_ratio)

    # Split the images into train and test sets
    train_images = images[:split_index]
    test_images = images[split_index:]

    # Move the train images to the train directory
    for image in train_images:
        src = os.path.join(member_path, image)
        dst = os.path.join(train_member_dir, image)
        shutil.copyfile(src, dst)

    # Move the test images to the test directory
    for image in test_images:
        src = os.path.join(member_path, image)
        dst = os.path.join(test_member_dir, image)
        shutil.copyfile(src, dst)

print("Dataset organized and split successfully!")

Dataset organized and split successfully!


In [33]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
import os

# Load the pre-trained VGG16 model
model = VGG16(weights='imagenet', include_top=False)

# Define a function to extract features from images
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))  # VGG16 input size
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = model.predict(img_data)
    return features.flatten()


# Initialize lists to store image features and corresponding labels
train_features = []
test_features = []
train_labels = []
test_labels = []

train_dir = "new_dataset1/train"
test_dir = "new_dataset1/test"

# Extract features from train images
for member_folder in os.listdir(train_dir):
    member_path = os.path.join(train_dir, member_folder)
    for image_file in os.listdir(member_path):
        image_path = os.path.join(member_path, image_file)
        features = extract_features(image_path, model)
        train_features.append(features)
        train_labels.append(member_folder)

# Extract features from test images
for member_folder in os.listdir(test_dir):
    member_path = os.path.join(test_dir, member_folder)
    for image_file in os.listdir(member_path):
        image_path = os.path.join(member_path, image_file)
        features = extract_features(image_path, model)
        test_features.append(features)
        test_labels.append(member_folder)

X_train = np.array(train_features)
X_test = np.array(test_features)
y_train = np.array(train_labels)
y_test = np.array(test_labels)

1/1 [==============================] - 1s 871ms/step


In [38]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

img_width, img_height = 150, 150
batch_size = 32

# Data Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')  # Change to 'categorical' if more than two classes

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')  # Change to 'categorical' if more than two classes

# Define the Model Architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='softmax'))  # Change to 'softmax' if more than two classes

# Compile the Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model
epochs = 5  # You can adjust the number of epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size)

# Evaluate the Model
loss, accuracy = model.evaluate(test_generator)
print("Test Accuracy:", accuracy)

Found 1332 images belonging to 9 classes.
Found 340 images belonging to 9 classes.
Epoch 1/5
41/41 [==============================] - 101s 2s/step - loss: 0.3587 - accuracy: 0.1111 - val_loss: 0.3492 - val_accuracy: 0.1111
Epoch 2/5
41/41 [==============================] - 109s 3s/step - loss: 0.3491 - accuracy: 0.1111 - val_loss: 0.3490 - val_accuracy: 0.1111
Epoch 3/5
41/41 [==============================] - 89s 2s/step - loss: 0.3491 - accuracy: 0.1111 - val_loss: 0.3492 - val_accuracy: 0.1111
Epoch 4/5
41/41 [==============================] - 89s 2s/step - loss: 0.3491 - accuracy: 0.1111 - val_loss: 0.3490 - val_accuracy: 0.1111
Epoch 5/5
11/11 [==============================] - 10s 916ms/step - loss: 0.3491 - accuracy: 0.1111
Test Accuracy: 0.1111111119389534
